## Main Notebook -> For Running Alignment (Stage-1 Training)

This notebook is used for alignment training of the models

In [1]:
%load_ext autoreload
%autoreload 2

In [80]:
from imports.configs.config import setup_from_yaml, ModelsConfig
# 🔹 Phase 0 – Global setup
cfg = setup_from_yaml("imports/configs/config.yaml")

device = cfg.torch_device
dtype = cfg.torch_dtype

print("Using device:", device)
print("Using dtype:", dtype)


[Config] Device: cuda, dtype: torch.float16
[Config] root_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass
[Config] features_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass/features
Using device: cuda
Using dtype: torch.float16


### Step-1: - Load the Encoded Dataset -> For Alignment

In [2]:
import json
import torch
from torch.utils.data import Dataset
from pathlib import Path

#### Step-1: - Load the Pixmo-Cap Dataset

In [3]:
import json
import torch
from torch.utils.data import Dataset
from pathlib import Path

class PixmoFeatureDataset(Dataset):
    """
    Loads the pre-extracted image features.
    It also fixes old paths like 'data/pixmo/...' -> 'data/data/pixmo/...'
    """
    def __init__(self, index_file: str | Path):
        index_file = Path(index_file)
        with open(index_file, "r") as f:
            self.index = json.load(f)

        # Base dir where your index lives (e.g. ./data/data/pixmo)
        self.base_dir = index_file.parent

    def _fix_path(self, raw_path: str) -> Path:
        p = Path(raw_path)

        # If it's already absolute and exists, just return
        if p.is_absolute() and p.exists():
            return p

        # Common case: path stored as "data/pixmo/features/xxx.pt"
        # but actual is "data/data/pixmo/features/xxx.pt"
        s = str(p)

        if "data/pixmo" in s and not p.exists():
            s = s.replace("data/pixmo", "data/data/pixmo")
            p2 = Path(s)
            if p2.exists():
                return p2

        # Otherwise, try resolving relative to the index directory
        p3 = (self.base_dir / p.name)  # fallback: same dir, same filename
        if p3.exists():
            return p3

        # Last resort: just return the original; will raise if missing
        return p

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        meta = self.index[idx]
        raw_path = meta["file"]
        file_path = self._fix_path(raw_path)

        blob = torch.load(file_path)

        return {
            "features": blob["features"],          # (num_patches, feat_dim)
            "caption": blob["caption"],            # raw caption text
            "num_patches": meta["num_patches"],
            "orig_idx": meta["orig_idx"],
        }


In [4]:
pixmo_train_ds = PixmoFeatureDataset("./data/data/pixmo/train_index.json")
pixmo_val_ds   = PixmoFeatureDataset("./data/data/pixmo/val_index.json")

print("PixMo train:", len(pixmo_train_ds))
print("PixMo val:", len(pixmo_val_ds))


PixMo train: 26366
PixMo val: 4376


In [5]:
# Inspect one example
ex = pixmo_train_ds[0]
print(ex["features"].shape, ex["caption"][:80])

torch.Size([256, 1536]) The image depicts a pixelated, retro-style space shooter game set against a nigh


#### Step-2: - Load the Audio Dataset

In [73]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

class LibriSpeechFeatureDataset(Dataset):
    """
    Loads pre-extracted Whisper features.
    Fixes broken paths like 'data/librispeech/...' -> 'data/data/librispeech/...'
    just like PixmoFeatureDataset does.
    """

    def __init__(self, index_file: str | Path):
        index_file = Path(index_file)
        with open(index_file, "r") as f:
            self.index = json.load(f)

        # Base directory where index.json lives
        self.base_dir = index_file.parent

    def _fix_path(self, raw_path: str) -> Path:
        """
        Try multiple strategies to fix incorrect dataset paths.
        1. Use raw path if absolute + exists
        2. Fix common 'data/librispeech' → 'data/data/librispeech'
        3. Try rewriting relative to index dir
        """
        p = Path(raw_path)

        # 1. If fully absolute and exists → OK
        if p.is_absolute() and p.exists():
            return p

        s = str(p)

        # 2. Common mismatch:
        #    raw: "data/librispeech/features/train_feat_123.pt"
        #    actual: "data/data/librispeech/features/train_feat_123.pt"
        if "data/librispeech" in s and not p.exists():
            s2 = s.replace("data/librispeech", "data/data/librispeech")
            p2 = Path(s2)
            if p2.exists():
                return p2

        # 3. Fallback: resolve relative to the index directory
        #    (useful if someone moved the index folder)
        p3 = self.base_dir / p.name
        if p3.exists():
            return p3

        # ❌ Last fallback → just return original (torch.load will raise)
        return p

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        meta = self.index[idx]

        raw_path = meta["file"]
        file_path = self._fix_path(raw_path)

        blob = torch.load(file_path)

        return {
            "features": blob["features"],      # (T_enc, d_audio)
            "text": blob["text"],
            "duration": blob["duration"],
            "sampling_rate": blob["sampling_rate"],
            "orig_idx": blob["orig_idx"],
        }


In [7]:
audio_train_ds = LibriSpeechFeatureDataset("./data/data/librispeech/train_index.json")

print("Loaded LibriSpeech feature dataset:", len(audio_train_ds))

ex = audio_train_ds[0]
print(ex["features"].shape, ex["text"][:100])


Loaded LibriSpeech feature dataset: 8677
torch.Size([1500, 512]) CONCERNING THE DISEASE THAT HEROD FELL INTO AND THE SEDITION WHICH THE JEWS RAISED THEREUPON


In [81]:
cfg

Config(paths=PathsConfig(root_dir='/storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass', features_dir='/storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass/features'), models=ModelsConfig(vision_model_name='openai/clip-vit-base-patch32', llm_model_name='qwen/Qwen2.5-3B-Instruct', audio_model_name='openai/whisper-base'), architecture=ArchitectureConfig(perceiver_dim=None, num_latents=64, num_perceiver_layers=4, num_attn_heads=8, mlp_ratio=4.0), training=TrainingConfig(batch_size=16, num_epochs=5, learning_rate='3e-4', weight_decay=0.01, warmup_steps=500, max_grad_norm=1.0, log_every_steps=50, train_subset_size=2000, val_subset_size=500), mrl=MRLConfig(mrl_dims=[1024, 512, 256], mrl_weight=1.0, mrl_temp=0.07), misc=MiscConfig(dtype='float16', seed=42, device='auto', use_wandb=True, wandb_project='edge_glass', wandb_run_name='phase0_global_setup'), torch_device=device(type='cuda'), torch_dtype=torch.float16)

### Build the dataset

In [ ]:
def collate_alignment(batch, tokenizer, device="cpu"):
    """
    batch: list of dicts with keys:
      - features: (L_i, D)
      - text: str
      - modality: "vision" or "audio"
    """
    # 1) Feature padding
    seqs = [b["features"] for b in batch]             # list of (L_i, D)
    lengths = [s.size(0) for s in seqs]
    max_len = max(lengths)
    feat_dim = seqs[0].size(1)
    B = len(batch)

    feats = torch.zeros(B, max_len, feat_dim, dtype=seqs[0].dtype)
    feat_mask = torch.zeros(B, max_len, dtype=torch.bool)

    for i, (s, L) in enumerate(zip(seqs, lengths)):
        feats[i, :L] = s
        feat_mask[i, :L] = True   # True where there is real data

    # 2) Text tokenization (for LLM)
    texts = [b["text"] for b in batch]
    tok = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=tokenizer.model_max_length,
        return_tensors="pt",
    )

    # 3) Modality as tensor of ints (0 = vision, 1 = audio), if you want
    modality_strs = [b.get("modality", "vision") for b in batch]
    modality_ids = torch.tensor(
        [0 if m == "vision" else 1 for m in modality_strs],
        dtype=torch.long,
    )

    batch_out = {
        "features": feats.to(device),                 # (B, max_L, D)
        "feature_mask": feat_mask.to(device),         # (B, max_L)
        "input_ids": tok["input_ids"].to(device),     # (B, T_text)
        "attention_mask": tok["attention_mask"].to(device),
        "modality_ids": modality_ids.to(device),      # (B,)
        "raw_text": texts,
    }
    return batch_out


In [ ]:
from functools import partial

vision_collate = partial(collate_alignment, tokenizer=llm_tokenizer, device=device)

pixmo_train_loader = DataLoader(
    pixmo_train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=vision_collate,
)

pixmo_val_loader = DataLoader(
    pixmo_val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    collate_fn=vision_collate,
)


In [ ]:
audio_train_loader = DataLoader(
    audio_train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=vision_collate,   # same collate
)


### Step-2:- Load the Architecture(Encoders + Alignment Layers)

In [8]:
from imports.encoders import VisionEncoder, AudioEncoder
from imports.perceiver import PerceiverLatentEncoder, ProjectorMLP
import torch, torch.nn as nn

/home/hice1/vchopra37/scratch/projects/edge_glass/edge_glass_env/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16  # or bfloat16 / float32 depending on your env

In [10]:
# 1) Load frozen encoders
vision_enc = VisionEncoder(
    model_name="facebook/dinov2-base",
    device=device,
    dtype=dtype,
)

# Use a real image if available
try:
    img1 = Image.open("/mnt/data/sample1.jpg")
    img2 = Image.open("/mnt/data/sample2.jpg")
    images = [img1, img2]
except:
    # fallback: create dummy RGB images of size 224x224
    import numpy as np
    from PIL import Image

    dummy = (np.random.rand(224,224,3) * 255).astype('uint8')
    images = [Image.fromarray(dummy), Image.fromarray(dummy)]
    
    
vision_out = vision_enc.encode_images(images)
vision_feats, vision_mask = vision_out["feats"], vision_out["mask"]

#### Test the encoders with random image and audio

In [11]:
audio_enc = AudioEncoder(
    model_name="openai/whisper-base",
    device=device,
    dtype=torch.float16,
)

# 1) (B, T) tensor
B, T = 2, 16000 * 3
waveforms = torch.randn(B, T)
sr = 16000

In [12]:
out = audio_enc.encode_waveforms(waveforms, sample_rates=sr)
print("Case 1 feats:", out["feats"].shape, "mask:", out["mask"].shape)

Case 1 feats: torch.Size([2, 1500, 512]) mask: torch.Size([2, 1500])


In [13]:
# 2) (B, 1, T) tensor
waveforms_3d = waveforms.unsqueeze(1)
out2 = audio_enc.encode_waveforms(waveforms_3d, sample_rates=sr)
print("Case 2 feats:", out2["feats"].shape, "mask:", out2["mask"].shape)

# 3) list[Tensor] with slightly different shapes (simulating variable length)
waveforms_list = [torch.randn(T), torch.randn(T // 2)]
out3 = audio_enc.encode_waveforms(waveforms_list, sample_rates=sr)
print("Case 3 feats:", out3["feats"].shape, "mask:", out3["mask"].shape)

Case 2 feats: torch.Size([2, 1500, 512]) mask: torch.Size([2, 1500])
Case 3 feats: torch.Size([2, 1500, 512]) mask: torch.Size([2, 1500])


#### Load the multimodal model

In [54]:
from imports.model import MultiModalAlignmentModel  # or from current cell
from PIL import Image
import torch
from torchviz import make_dot

In [55]:
model = MultiModalAlignmentModel(
    d_shared=512,
    d_latent=512,
    d_align=1024,
    num_latents=32,   # smaller for viz
    num_layers=2,
    num_heads=4,
    use_perceiver=True,
    dtype=torch.float32,
    device=device,
)
model.eval()


MultiModalAlignmentModel(
  (vision_encoder): VisionEncoder(
    (model): Dinov2Model(
      (embeddings): Dinov2Embeddings(
        (patch_embeddings): Dinov2PatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): Dinov2Encoder(
        (layer): ModuleList(
          (0-11): 12 x Dinov2Layer(
            (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (attention): Dinov2Attention(
              (attention): Dinov2SelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
              )
              (output): Dinov2SelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace

In [83]:
def describe(name, t):
    if t is None:
        print(f"{name}: None")
    else:
        print(f"{name}: shape={tuple(t.shape)}, dtype={t.dtype}, "
              f"mean={t.mean().item():.4f}, std={t.std().item():.4f}")


In [84]:
B = 2

# Vision: dummy pixel values (already "preprocessed"-ish tensor)
dummy_images = torch.randn(B, 3, 224, 224, device=device)

# Audio: dummy raw waveforms (e.g. 3 seconds at 16kHz)
dummy_waveforms = torch.randn(B, 16000 * 3, device=device)
dummy_sr = 16000


In [85]:
with torch.no_grad():
    # ---- 1) Encoder ----
    enc_v = model.vision_encoder.encode_images(dummy_images)
    v_feats = enc_v["feats"].to(device)   # (B, T_v, D_v)
    v_mask  = enc_v["mask"]              # (B, T_v) bool

    describe("vision/encoder_feats", v_feats)
    describe("vision/mask", v_mask.float())

    # ---- 2) Adapter to shared dim ----
    model._ensure_vision_adapter(v_feats.size(-1))
    v_shared = model.vision_adapter(v_feats)   # (B, T_v, d_shared)
    describe("vision/shared_feats (after adapter)", v_shared)

    # ---- 3) Perceiver latents ----
    if model.use_perceiver:
        v_latents = model.perceiver(v_shared, encoder_mask=v_mask)  # (B, L, d_latent)
        describe("vision/perceiver_latents", v_latents)

        # ---- 4) Projector (token-level) ----
        v_tokens = model.projector(v_latents)                        # (B, L, d_align)
        describe("vision/tokens_after_projector", v_tokens)

        # ---- 5) Pooled alignment embedding ----
        v_pooled = v_tokens.mean(dim=1)                              # (B, d_align)
        describe("vision/pooled", v_pooled)
    else:
        print("Perceiver disabled; using pooled-only path.")
        v_pooled_in = model._pool_masked_mean(v_shared, v_mask)
        v_pooled = model.projector(v_pooled_in)
        v_latents = None
        v_tokens = None
        describe("vision/pooled", v_pooled)


vision/encoder_feats: shape=(2, 256, 1536), dtype=torch.float32, mean=-0.0171, std=1.1842
vision/mask: shape=(2, 256), dtype=torch.float32, mean=1.0000, std=0.0000
vision/shared_feats (after adapter): shape=(2, 256, 512), dtype=torch.float32, mean=0.0004, std=0.6689
vision/perceiver_latents: shape=(2, 32, 512), dtype=torch.float32, mean=0.0417, std=0.5438
vision/tokens_after_projector: shape=(2, 32, 1024), dtype=torch.float32, mean=0.0071, std=0.1993
vision/pooled: shape=(2, 1024), dtype=torch.float32, mean=0.0071, std=0.1986


In [86]:
with torch.no_grad():
    # ---- 1) Encoder ----
    enc_a = model.audio_encoder.encode_waveforms(dummy_waveforms, sample_rates=dummy_sr)
    a_feats = enc_a["feats"].to(device)   # (B, T_a, D_a)
    a_mask  = enc_a["mask"]              # (B, T_a) bool

    describe("audio/encoder_feats", a_feats)
    describe("audio/mask", a_mask.float())

    # ---- 2) Adapter to shared dim ----
    model._ensure_audio_adapter(a_feats.size(-1))
    a_shared = model.audio_adapter(a_feats)   # (B, T_a, d_shared)
    describe("audio/shared_feats (after adapter)", a_shared)

    # ---- 3) Perceiver latents ----
    if model.use_perceiver:
        a_latents = model.perceiver(a_shared, encoder_mask=a_mask)  # (B, L, d_latent)
        describe("audio/perceiver_latents", a_latents)

        # ---- 4) Projector (token-level) ----
        a_tokens = model.projector(a_latents)                        # (B, L, d_align)
        describe("audio/tokens_after_projector", a_tokens)

        # ---- 5) Pooled alignment embedding ----
        a_pooled = a_tokens.mean(dim=1)                              # (B, d_align)
        describe("audio/pooled", a_pooled)
    else:
        print("Perceiver disabled; using pooled-only path.")
        a_pooled_in = model._pool_masked_mean(a_shared, a_mask)
        a_pooled = model.projector(a_pooled_in)
        a_latents = None
        a_tokens = None
        describe("audio/pooled", a_pooled)


audio/encoder_feats: shape=(2, 1500, 512), dtype=torch.float32, mean=-0.0190, std=1.3926
audio/mask: shape=(2, 1500), dtype=torch.float32, mean=1.0000, std=0.0000
audio/shared_feats (after adapter): shape=(2, 1500, 512), dtype=torch.float32, mean=-0.0090, std=0.7969
audio/perceiver_latents: shape=(2, 32, 512), dtype=torch.float32, mean=-0.0019, std=0.5730
audio/tokens_after_projector: shape=(2, 32, 1024), dtype=torch.float32, mean=0.0041, std=0.1987
audio/pooled: shape=(2, 1024), dtype=torch.float32, mean=0.0041, std=0.1985


In [87]:
with torch.no_grad():
    v_enc_api = model.encode_vision(dummy_images)
    a_enc_api = model.encode_audio(dummy_waveforms, dummy_sr)

describe("encode_vision()['tokens']", v_enc_api["tokens"])
describe("encode_vision()['pooled']", v_enc_api["pooled"])

describe("encode_audio()['tokens']", a_enc_api["tokens"])
describe("encode_audio()['pooled']", a_enc_api["pooled"])


encode_vision()['tokens']: shape=(2, 32, 1024), dtype=torch.float32, mean=0.0071, std=0.1993
encode_vision()['pooled']: shape=(2, 1024), dtype=torch.float32, mean=0.0071, std=0.1986
encode_audio()['tokens']: shape=(2, 32, 1024), dtype=torch.float32, mean=0.0041, std=0.1987
encode_audio()['pooled']: shape=(2, 1024), dtype=torch.float32, mean=0.0041, std=0.1985


### Plotting the Architecture

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from imports.model import FullAlignmentGraphWrapper


wrapper = FullAlignmentGraphWrapper(model).to(device)
wrapper.eval()


FullAlignmentGraphWrapper(
  (core): MultiModalAlignmentModel(
    (vision_encoder): VisionEncoder(
      (model): Dinov2Model(
        (embeddings): Dinov2Embeddings(
          (patch_embeddings): Dinov2PatchEmbeddings(
            (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): Dinov2Encoder(
          (layer): ModuleList(
            (0-11): 12 x Dinov2Layer(
              (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
              (attention): Dinov2Attention(
                (attention): Dinov2SelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                )
                (output): Dinov2SelfOutput(
                  (dense): Linear(in_features=768, o

In [61]:
# Vision: dummy pixel_values (B, 3, H, W)
dummy_pixel_values = torch.randn(2, 3, 224, 224, device=device)

# Audio: dummy raw waveforms (B, T)
dummy_waveforms = torch.randn(2, 16000 * 3, device=device)  # 3 seconds @ 16kHz


In [62]:
with torch.no_grad():
    z = wrapper(dummy_pixel_values, dummy_waveforms)

print("Output shape:", z.shape)


Output shape: torch.Size([2, 1024])


In [76]:
from torchview import draw_graph

graph = draw_graph(
    wrapper,
    input_data=(dummy_pixel_values, dummy_waveforms),
    graph_name="EdgeGlassAlignmentFull",
    expand_nested=True,
    depth=3,                 # increase to 4 for more detail
    save_graph=True,
    directory="arch_plots",
    filename="edgeglass_alignment_full_graph",
)

# graph.visual_graph

### Step-3: - Alignment Training

In [77]:
from imports.align_training.steps import AlignmentModules, AlignmentConfig
from imports.align_training.training import (
    build_alignment_optimizer,
    train_alignment,
)

In [ ]:
vision_loader = DataLoader(
    pixmo_train_ds,
    batch_size=cfg.batch_size,
    shuffle=True,
    collate_fn=collate_features_with_text,  # returns encoder_feats, encoder_mask, texts
)

audio_loader = DataLoader(
    audio_train_ds,
    batch_size=cfg.batch_size,
    shuffle=True,
    collate_fn=collate_features_with_text,
)

train_loaders = {
    "vision": vision_loader,
    "audio": audio_loader,
}


In [ ]:
trainable_modules = torch.nn.Module()
trainable_modules.vision_adapter = vision_adapter
trainable_modules.audio_adapter  = audio_adapter
trainable_modules.perceiver      = perceiver
trainable_modules.projector      = projector

optimizer = build_alignment_optimizer(
    trainable_modules=trainable_modules,
    learning_rate=cfg.learning_rate,
    weight_decay=cfg.weight_decay,
)


In [ ]:


# You already have cfg, vision_adapter, audio_adapter, perceiver, projector, device

modules = AlignmentModules(
    vision_adapter=vision_adapter,
    audio_adapter=audio_adapter,
    perceiver=perceiver,
    projector=projector,
)

align_cfg = AlignmentConfig(
    mrl_dims=tuple(cfg.mrl_dims),
    mrl_temperature=cfg.mrl_temperature,
    max_text_length=64,
)


NameError: name 'vision_adapter' is not defined

#### Training Loop

In [ ]:

train_alignment(
    train_loaders=train_loaders,
    modules=modules,
    cfg=align_cfg,
    text_embed_fn=text_embed_fn,
    optimizer=optimizer,
    device=device,
    num_epochs=cfg.num_epochs,
    log_every=50,
    log_fn=wandb_log_fn,
    modalities=("vision", "audio"),
)